### purpose

train RDA on complex sims - no RDA outliers

first create some directories and files with python then use R to estimate offset

# Create muts files

In [1]:
from pythonimports import *

import MVP_01_train_gradient_forests as mvp01
import MVP_summary_functions as mvp

# lview, dview = get_client(cluster_id='1678456057-hrmc', profile='lotterhos')

tdir = '/home/b.lind/offsets/run_20220919_tutorial'

outerdir = '/home/b.lind/offsets/run_20220919_tutorial'
tdir = '/home/b.lind/offsets/run_20220919_tutorial/tutorial'

gf_training_dir = op.join(outerdir, 'gradient_forests/training/training_files')
gf_outdir = gf_training_dir.replace('/training_files', '/training_outfiles')
gf_shdir = gf_training_dir.replace('/training_files', '/training_shfiles')
rona_training_dir = gf_training_dir.replace('/gradient_forests/', '/RONA/')

rda_dir = makedir(op.join(outerdir, 'rda'))
catdir = makedir(op.join(rda_dir, 'rda_catfiles'))
shdir = makedir(op.join(rda_dir, 'shfiles'))

mvp.latest_commit()
session_info.show()

#########################################################
Today:	March 21, 2023 - 12:43:36
python version: 3.8.5

Current commit of pythonimports:
commit 8d4f83fd91caa2e58acbafcadfde07d19ae79379  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Fri Mar 3 00:48:36 2023 -0500

Current commit of MVP_offsets:
commit 9a9063c95958d4721fef27d4a0f317c192ed25d3  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Mon Mar 20 09:54:43 2023 -0400
#########################################################



# look at example mutsfile

In [2]:
mvp01.slimdir = mvp.slimdir
mvp01.seed = '1231094'
muts = mvp01.read_muts_file()

muts[['causal_temp', 'causal_sal']]


Reading muts file ...


,causal_temp,causal_sal
1-9,neutral-linked,neutral-linked
1-22,neutral-linked,neutral-linked
1-46,neutral-linked,neutral-linked
1-110,neutral-linked,neutral-linked
1-141,neutral-linked,neutral-linked
...,...,...
20-999859,neutral,neutral
20-999873,neutral,neutral
20-999875,neutral,neutral
20-999883,neutral,neutral


In [3]:
muts.causal_temp.value_counts()

neutral-linked    12886
neutral           12867
causal              310
Name: causal_temp, dtype: int64

# create muts file for RDA scripts

In [4]:
snps = pd.read_table(op.join(tdir, 'SNPs.txt'), delim_whitespace=True)

print(snps.shape)

snps.head()

(26371, 5)


,LG,pos_pyslim,mutname,a_freq_subset,causal
0,1,8,1-8,0.0175,False
1,1,27,1-27,0.0120,False
2,1,34,1-34,0.0370,False
3,1,81,1-81,0.0330,False
4,1,97,1-97,0.0170,False


In [5]:
# from 00_create_files_for_GF_and_RONA_train_GF.ipynb
marker_sets = {
    'all' : snps.mutname.tolist(),
    'neutral' : snps[snps.LG > 10]['mutname'].tolist(),
    'adaptive' : snps[snps.causal == True]['mutname'].tolist()
}

for marker_set, loci in marker_sets.items():
    print(marker_set, len(loci))

all 26371
neutral 13285
adaptive 90


In [6]:
# label adaptive loci
snps['mutID'] = [0 if snp in marker_sets['adaptive'] else 1 for snp in snps['mutname'] ]  # RDA script looks for: muts[, 'mutID'] != 1

# add columns for neutral (script assumed 2 traits, but if it's neutral it's neutral, eh?)
# rda expects 'neutral' entries in both columns if a marker is neutral
snps['causal_temp'] = ['causal' if snp in marker_sets['adaptive'] else 'neutral' for snp in snps['mutname']]
snps.loc[(snps['LG'] <= 10) & (snps['mutID']!=0), 'causal_temp'] = 'neutral-linked'
snps.loc[(snps['mutID']==0), 'causal_temp'] = 'causal'
snps['causal_sal'] = snps['causal_temp'].copy()

In [7]:
# non-adaptive snps have non-zero mutIDs
snps['mutID'][(snps.causal_temp == 'neutral') & (snps.causal_sal == 'neutral')].value_counts()

1    13285
Name: mutID, dtype: int64

In [8]:
snps.causal_temp.value_counts()

neutral           13285
neutral-linked    12996
causal               90
Name: causal_temp, dtype: int64

In [9]:
sum(snps.mutID == 0), len(marker_sets['adaptive'])

(90, 90)

### save

In [10]:
# files used in MVP_12_RDA_offset.R
#  paste0(slimdir, sprintf('/%s_Rout_muts_full.txt', seed))
#
#  paste0(
#             paste0(outerdir, '/pca/mutfiles'),
#             sprintf('/%s_pooled_Rout_muts_full.txt', seed)
#         )

In [11]:
rda_slimdir = makedir(op.join(rda_dir, 'rda_slimdir'))

mutsfile = op.join(rda_slimdir, 'tutorial_Rout_muts_full.txt')

snps.to_csv(mutsfile, index=False, header=True, sep='\t')

In [12]:
mutsfile

'/home/b.lind/offsets/run_20220919_tutorial/rda/rda_slimdir/tutorial_Rout_muts_full.txt'

In [13]:
pd.read_table(mutsfile)

,LG,pos_pyslim,mutname,a_freq_subset,causal,mutID,causal_temp,causal_sal
0,1,8,1-8,0.0175,False,1,neutral-linked,neutral-linked
1,1,27,1-27,0.0120,False,1,neutral-linked,neutral-linked
2,1,34,1-34,0.0370,False,1,neutral-linked,neutral-linked
3,1,81,1-81,0.0330,False,1,neutral-linked,neutral-linked
4,1,97,1-97,0.0170,False,1,neutral-linked,neutral-linked
...,...,...,...,...,...,...,...,...
26366,20,999742,20-999742,0.0125,False,1,neutral,neutral
26367,20,999764,20-999764,0.0120,False,1,neutral,neutral
26368,20,999823,20-999823,0.0120,False,1,neutral,neutral
26369,20,999845,20-999845,0.0135,False,1,neutral,neutral


In [ ]:
# outerdir

In [ ]:
# # symlink
# dst_dir = makedir(op.join(outerdir, 'pca/mutfiles'))
# dst = op.join(dst_dir, 'tutorial_pooled_Rout_muts_full.txt')

# try:
#     os.symlink(mutsfile, dst)
# except FileExistsError:
#     pass

In [ ]:
# dst

# Get PCA data

Run MVP_pooled_pca_and_rda.R but with tweaks for 6 traits

In [1]:
options(run.main=FALSE)
source('/home/b.lind/code/MVP-offsets/01_src/MVP_pooled_pca_and_rda.R')
source('/home/b.lind/code/MVP-offsets/01_src/MVP_12_RDA_offset.R')

Loading required package: permute

Loading required package: lattice


Attaching package: ‘lattice’


The following object is masked from ‘package:LEA’:

    barchart


This is vegan 2.6-2

Loading required package: fit.models



R version 4.0.3 (2020-10-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /home/b.lind/anaconda3/envs/MVP_env_R4.0.3/lib/libopenblasp-r0.3.17.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] progress_1.2.2    qvalue_2.22.0     robust_0.7-0      fit.models_0.64  
[5] vegan_2.6-2       lattice_0.20-45   permute_0.9-7     data.table_1.14.2
[9] LEA_3.8.0        

loaded via a namespace (and not attached):
 [1] pbdZMQ_0.3-7      repr_1.1.4        reshape2_1.4.4    splines_4.0.3    

In [2]:
# similar to MVP_pooled_pca_and_rda.R::run_rda
    # but for 6 traits
run_rda = function(snps){
    #----------------------------------------------------------------------------------------------------#
    # Run structure-corrected and -uncorrected RDA.
    # 
    # Parameters
    # ----------
    # snps - data.frame; output from `prep_snps` - pops for rows, loci for cols, frequencies for entries
    #
    # Notes
    # -----
    # - structure correction is done from PCs estimated using all loci
    #----------------------------------------------------------------------------------------------------#

    # get current environmental data
    env_pres = get_curr_envdata(ntraits=6)
    env1_mat = env_pres[, 'env1_mat']
    env2_MTWetQ = env_pres[, 'env2_MTWetQ']
    env3_MTDQ = env_pres[, 'env3_MTDQ']
    env4_PDM = env_pres[, 'env4_PDM']
    env5_PwarmQ = env_pres[, 'env5_PwarmQ']
    env6_PWM = env_pres[, 'env6_PWM']
    
    # read in PCA data
    pcdata = get_pc_data(env_pres)
    
    # structure-uncorrected RDA
    rdaout <- rda(snps ~ env1_mat + env2_MTWetQ + env3_MTDQ + env4_PDM + env5_PwarmQ + env6_PWM)
    
    # structure-corrected RDA
    rdaout_corr = rda(
        snps ~ env1_mat + env2_MTWetQ + env3_MTDQ + env4_PDM + env5_PwarmQ + env6_PWM + Condition(pcdata[, 'PC1'],
                                                                                                  pcdata[, 'PC2'])
    )
    
    # save
    rda_outdir = paste0(outerdir, '/rda/rda_files')
    if (dir.exists(rda_outdir) == FALSE){
        dir.create(rda_outdir, recursive=TRUE)
    }
    file = paste0(
        rda_outdir,
        sprintf('/%s_pooled_RDA.RDS', seed)
    )
    file_corr = paste0(
        rda_outdir,
        sprintf('/%s_pooled_RDA_structcorr.RDS', seed)
    )
    
    saveRDS(rdaout, file)
    saveRDS(rdaout_corr, file_corr)

    # create a muts file like Katie's
    create_muts_file(rdaout, rdaout_corr)
    
}

In [3]:
seed <<- 'tutorial'
slimdir <<- '/home/b.lind/offsets/run_20220919_tutorial/rda/rda_slimdir'
snpfile <<- '/home/b.lind/offsets/run_20220919_tutorial/gradient_forests/training/training_files/tutorial_Rout_Gmat_sample_maf-gt-p01_GFready_pooled_all.txt'
outerdir <<- '/home/b.lind/offsets/run_20220919_tutorial'
mvp_dir <<- '/home/b.lind/code/MVP-offsets/01_src'
ind_or_pooled <<- 'pooled'

In [4]:
snps = run_pca()

[1] "Performing PCA ..."
[1] "******************************"
[1] " Principal Component Analysis "
[1] "******************************"
summary of the options:

        -n (number of individuals)          100
        -L (number of loci)                 26371
        -K (number of principal components) 30
        -x (genotype file)                  /work/lotterhos/MVP-Offsets/run_20220919_tutorial/pca/pca_output/tutorial_Rout_Gmat_sample_maf-gt-p01_GFready_pooled_all_for_pca.lfmm
        -a (eigenvalue file)                /work/lotterhos/MVP-Offsets/run_20220919_tutorial/pca/pca_output/tutorial_Rout_Gmat_sample_maf-gt-p01_GFready_pooled_all_for_pca.pca/tutorial_Rout_Gmat_sample_maf-gt-p01_GFready_pooled_all_for_pca.eigenvalues
        -e (eigenvector file)               /work/lotterhos/MVP-Offsets/run_20220919_tutorial/pca/pca_output/tutorial_Rout_Gmat_sample_maf-gt-p01_GFready_pooled_all_for_pca.pca/tutorial_Rout_Gmat_sample_maf-gt-p01_GFready_pooled_all_for_pca.eigenvectors
        -

In [5]:
gf_training_dir = '/home/b.lind/offsets/run_20220919_tutorial/gradient_forests/training/training_files'
run_rda(snps)



Reading in current environmental data ...

saved pooled muts file to: /home/b.lind/offsets/run_20220919_tutorial/pca/mutfiles/tutorial_pooled_Rout_muts_full.txt

In [6]:
read.table('/home/b.lind/offsets/run_20220919_tutorial/pca/mutfiles/tutorial_pooled_Rout_muts_full.txt', header=T)

RDA1_score,RDA2_score,RDA1_score_corr,RDA2_score_corr,RDA_mlog10P,RDA_mlog10P_sig,RDA_mlog10P_corr,RDA_mlog10P_sig_corr,mutname
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,<lgl>,<chr>
0.003449311,-0.018104093,-1.534792e-02,-0.004312443,0.55714882,FALSE,0.442990874,FALSE,1-8
0.009940909,-0.005182073,-1.250555e-02,-0.016516572,0.13422519,FALSE,0.836949037,FALSE,1-27
0.008242055,0.012383775,2.523032e-02,-0.034138435,0.30123430,FALSE,1.972638455,FALSE,1-34
-0.032613740,0.016952612,1.961240e-02,-0.005458778,1.99574361,FALSE,0.389061521,FALSE,1-81
-0.016935113,-0.003119004,-3.518722e-04,0.002692840,0.51148603,FALSE,0.004713782,FALSE,1-97
-0.031782110,0.010584943,1.790724e-02,-0.010646350,1.67091727,FALSE,0.415392371,FALSE,1-153
0.001752268,-0.009215456,-1.070860e-02,0.014087866,0.15047232,FALSE,0.311667566,FALSE,1-154
0.008904909,0.013396100,2.745189e-02,-0.036963655,0.35677979,FALSE,2.311137978,FALSE,1-206
0.008625344,-0.023020008,-2.398358e-02,-0.006519803,0.93513170,FALSE,0.984823921,FALSE,1-287


# estimate offset in R

use basic functionality of MVP_12_RDA_offset.R

In [1]:
options(run.main=FALSE)
source('/home/b.lind/code/MVP-offsets/01_src/MVP_12_RDA_offset.R')

Loading required package: permute

Loading required package: lattice


Attaching package: ‘lattice’


The following object is masked from ‘package:LEA’:

    barchart


This is vegan 2.6-2



R version 4.0.3 (2020-10-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /home/b.lind/anaconda3/envs/MVP_env_R4.0.3/lib/libopenblasp-r0.3.17.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] vegan_2.6-2       lattice_0.20-45   permute_0.9-7     LEA_3.8.0        
[5] progress_1.2.2    data.table_1.14.2

loaded via a namespace (and not attached):
 [1] pillar_1.7.0      compiler_4.0.3    prettyunits_1.1.1 base64enc_0.1-3  
 [5] tools_4.0.3       digest_0.6.29     uuid_1.1-0      

In [2]:
outerdir <<- '/home/b.lind/offsets/run_20220919_tutorial'
seed <<- 'tutorial'
ind_or_pooled <<- 'pooled'
slimdir <<- paste0(outerdir, '/rda/rda_slimdir')
gf_training_dir <<- paste0(outerdir, '/gradient_forests/training/training_files')


In [3]:
# created in previous section
rda_files = list.files(paste0(outerdir, '/rda/rda_files'), full.names=T)

rda_files

[1] "/home/b.lind/offsets/run_20220919_tutorial/rda/rda_files/tutorial_pooled_RDA_structcorr.RDS"
[2] "/home/b.lind/offsets/run_20220919_tutorial/rda/rda_files/tutorial_pooled_RDA.RDS"

# functions

In [4]:
# overwrite function from MVP_12_RDA_offset.R
get_garden_climates <- function(x){
    # garden_climates is same thing as env_pres (as long as pooled)
    return(data.frame(env_pres))
}

In [5]:
# overwrite function from MVP_12_RDA_offset.R
train_outlier_rda = function(outlier_snps, rda, env_pres, ntraits=6){
    #---------------------------------------------------------------#
    # Train new RDA on outliers inferred from script input args.
    # 
    # Parameters
    # ----------
    # outlier_snps
    #     - output from subset_snps
    # rda
    #     - object loaded from rda_file input arg to this script
    # ntraits
    #     - the number of traits (environments) under selection in the simulation seed
    # 
    # Notes
    # -----
    # - assumes any structure correction is done with first two PCs
    #     - see get_pc_data()
    #---------------------------------------------------------------#
    cat(sprintf('\n\nTraining outlier RDA ...'))

    # decide if I need to estimate a new RDA
    if (use_RDA_outliers == FALSE & ntraits==6){
        # just use the original RDA object - this could be structure-corrected or not
        outlier_rda = rda

    } else {  # estimate a new RDA object with `outlier_snps`

        if (grepl('_RDA_structcorr.RDS', basename(rda_file))){  # this is structure corrected
            # get PCs
            pcdata = get_pc_data(env_pres)

            # replicate Variables object from capblancq & forester
            Variables = data.frame(pcdata, env_pres)

            # get RDA
            outlier_rda = rda(
                outlier_snps ~ env1_mat + env2_MTWetQ + env3_MTDQ + env4_PDM + env5_PwarmQ + env6_PWM + Condition(PC1 + PC2),
                Variables
            )


        } else {  # this is not structure corrected
            # replicate Variables object from capblancq & forester
            Variables = data.frame(env_pres)

            # get RDA
            outlier_rda = rda(
                outlier_snps ~ env1_mat + env2_MTWetQ + env3_MTDQ + env4_PDM + env5_PwarmQ + env6_PWM,
                Variables
            )
        }
    }

    return(outlier_rda)
}

In [6]:
# rewrite main function from MVP_12_RDA_offset.R
main <- function(use_RDA_outliers, rda_file){
    print(c(use_RDA_outliers, rda_file))
    
    # read in RDA object trained on all loci (either structure-corrected or not)
#     rda_file <<- run_rda(outlier_snps)  # rda_file needed for MVP_12_RDA_offset.R functions
    rda = readRDS(rda_file)
    
    # get the SNPs, subset for outliers (which would be all snps if use_RDA_outliers==FALSE)
    outlier_snps = subset_snps()
    print(c('len(outlier_snps) = ', len(outlier_snps)))

    
    # below here is basically MVP_12_RDA_offset.R::offset_estimation
    
    # get current environmental data (created from 02.05.00_create_files_for_GF_and_RONA_train_GF.ipynb)
    env_pres <<- get_curr_envdata(ntraits=6)
    
    # train RDA on outliers
    outlier_rda = train_outlier_rda(outlier_snps, rda, env_pres, ntraits=6)
    
    # estimate RDA offset for each of the gardens on the simulated landscape
        # 'None' for `subset` because `subset` is passed to estimate_garden_offsets::get_garden_climates
    estimate_garden_offsets('None', outlier_rda, env_pres, ntraits=6)
    
}

# loop over loci sets

In [7]:
for (rda_file in rda_files){
    for (use_RDA_outliers in c('FALSE', 'CAUSAL', 'NEUTRAL', 'TRUE')){

        main(use_RDA_outliers, rda_file)
    }
}


[1] "FALSE"                                                                                      
[2] "/home/b.lind/offsets/run_20220919_tutorial/rda/rda_files/tutorial_pooled_RDA_structcorr.RDS"


Reading in SNPs ...
[1] "mutsfile ="                                                                                
[2] "/home/b.lind/offsets/run_20220919_tutorial/pca/mutfiles/tutorial_pooled_Rout_muts_full.txt"
[1] "len(outliers) = " "26371"           
[1] "len(outlier_snps) = " "26371"               


Reading in current environmental data ...

Training outlier RDA ...


estimating offset across gardens [==>----------------------------------------] 6/100 (  6%) eta:  3s

estimating offset across gardens [==>----------------------------------------] 7/100 (  7%) eta:  4s

estimating offset across gardens [==>----------------------------------------] 8/100 (  8%) eta:  4s

estimating offset across gardens [===>---------------------------------------] 9/100 (  9%) eta:  4s

estimating offset across gardens [===>--------------------------------------] 10/100 ( 10%) eta:  4s

estimating offset across gardens [====>-------------------------------------] 11/100 ( 11%) eta:  4s

estimating offset across gardens [====>-------------------------------------] 12/100 ( 12%) eta:  4s

estimating offset across gardens [====>-------------------------------------] 13/100 ( 13%) eta:  4s

estimating offset across gardens [=====>------------------------------------] 14/100 ( 14%) eta:  4s

estimating offset across gardens [=====>------------------------------------] 15/


estimating offset across gardens [====================================>-----] 87/100 ( 87%) eta:  1s

estimating offset across gardens [====================================>-----] 88/100 ( 88%) eta:  1s

estimating offset across gardens [====================================>-----] 89/100 ( 89%) eta:  0s

estimating offset across gardens [=====================================>----] 90/100 ( 90%) eta:  0s

estimating offset across gardens [=====================================>----] 91/100 ( 91%) eta:  0s

estimating offset across gardens [======================================>---] 92/100 ( 92%) eta:  0s

estimating offset across gardens [======================================>---] 93/100 ( 93%) eta:  0s

estimating offset across gardens [======================================>---] 94/100 ( 94%) eta:  0s

estimating offset across gardens [=======================================>--] 95/100 ( 95%) eta:  0s

estimating offset across gardens [=======================================>--] 96/



wrote offset to: /home/b.lind/offsets/run_20220919_tutorial/rda/offset_outfiles/tutorial_pooled_FALSE_ntraits-6_structcorr_rda_offset.txt

wrote rda list to: /home/b.lind/offsets/run_20220919_tutorial/rda/offset_outfiles/tutorial_pooled_FALSE_ntraits-6_structcorr_rda_list.RDS[1] "CAUSAL"                                                                                     
[2] "/home/b.lind/offsets/run_20220919_tutorial/rda/rda_files/tutorial_pooled_RDA_structcorr.RDS"


Reading in SNPs ...
[1] "mutsfile ="                                                                            
[2] "/home/b.lind/offsets/run_20220919_tutorial/rda/rda_slimdir/tutorial_Rout_muts_full.txt"
[1] "len(outliers) = " "90"              
[1] "len(outlier_snps) = " "90"                  


Reading in current environmental data ...

Training outlier RDA ...


estimating offset across gardens [==>----------------------------------------] 6/100 (  6%) eta:  3s

estimating offset across gardens [==>----------------------------------------] 7/100 (  7%) eta:  3s

estimating offset across gardens [==>----------------------------------------] 8/100 (  8%) eta:  4s

estimating offset across gardens [===>---------------------------------------] 9/100 (  9%) eta:  4s

estimating offset across gardens [===>--------------------------------------] 10/100 ( 10%) eta:  4s

estimating offset across gardens [====>-------------------------------------] 11/100 ( 11%) eta:  4s

estimating offset across gardens [====>-------------------------------------] 12/100 ( 12%) eta:  4s

estimating offset across gardens [====>-------------------------------------] 13/100 ( 13%) eta:  4s

estimating offset across gardens [=====>------------------------------------] 14/100 ( 14%) eta:  4s

estimating offset across gardens [=====>------------------------------------] 15/

...


estimating offset across gardens [====================================>-----] 87/100 ( 87%) eta:  1s

estimating offset across gardens [====================================>-----] 88/100 ( 88%) eta:  1s

estimating offset across gardens [====================================>-----] 89/100 ( 89%) eta:  0s

estimating offset across gardens [=====================================>----] 90/100 ( 90%) eta:  0s

estimating offset across gardens [=====================================>----] 91/100 ( 91%) eta:  0s

estimating offset across gardens [======================================>---] 92/100 ( 92%) eta:  0s

estimating offset across gardens [======================================>---] 93/100 ( 93%) eta:  0s

estimating offset across gardens [======================================>---] 94/100 ( 94%) eta:  0s

estimating offset across gardens [=======================================>--] 95/100 ( 95%) eta:  0s

estimating offset across gardens [=======================================>--] 96/



wrote offset to: /home/b.lind/offsets/run_20220919_tutorial/rda/offset_outfiles/tutorial_pooled_NEUTRAL_ntraits-6_nocorr_rda_offset.txt

wrote rda list to: /home/b.lind/offsets/run_20220919_tutorial/rda/offset_outfiles/tutorial_pooled_NEUTRAL_ntraits-6_nocorr_rda_list.RDS[1] "TRUE"                                                                            
[2] "/home/b.lind/offsets/run_20220919_tutorial/rda/rda_files/tutorial_pooled_RDA.RDS"


Reading in SNPs ...
[1] "mutsfile ="                                                                                
[2] "/home/b.lind/offsets/run_20220919_tutorial/pca/mutfiles/tutorial_pooled_Rout_muts_full.txt"
[1] "len(outliers) = " "2538"            
[1] "len(outlier_snps) = " "2538"                


Reading in current environmental data ...

Training outlier RDA ...


estimating offset across gardens [==>----------------------------------------] 6/100 (  6%) eta:  4s

estimating offset across gardens [==>----------------------------------------] 7/100 (  7%) eta:  4s

estimating offset across gardens [==>----------------------------------------] 8/100 (  8%) eta:  4s

estimating offset across gardens [===>---------------------------------------] 9/100 (  9%) eta:  4s

estimating offset across gardens [===>--------------------------------------] 10/100 ( 10%) eta:  4s

estimating offset across gardens [====>-------------------------------------] 11/100 ( 11%) eta:  4s

estimating offset across gardens [====>-------------------------------------] 12/100 ( 12%) eta:  4s

estimating offset across gardens [====>-------------------------------------] 13/100 ( 13%) eta:  4s

estimating offset across gardens [=====>------------------------------------] 14/100 ( 14%) eta:  4s

estimating offset across gardens [=====>------------------------------------] 15/


estimating offset across gardens [====================================>-----] 87/100 ( 87%) eta:  1s

estimating offset across gardens [====================================>-----] 88/100 ( 88%) eta:  1s

estimating offset across gardens [====================================>-----] 89/100 ( 89%) eta:  1s

estimating offset across gardens [=====================================>----] 90/100 ( 90%) eta:  0s

estimating offset across gardens [=====================================>----] 91/100 ( 91%) eta:  0s

estimating offset across gardens [======================================>---] 92/100 ( 92%) eta:  0s

estimating offset across gardens [======================================>---] 93/100 ( 93%) eta:  0s

estimating offset across gardens [======================================>---] 94/100 ( 94%) eta:  0s

estimating offset across gardens [=======================================>--] 95/100 ( 95%) eta:  0s

estimating offset across gardens [=======================================>--] 96/



wrote offset to: /home/b.lind/offsets/run_20220919_tutorial/rda/offset_outfiles/tutorial_pooled_TRUE_ntraits-6_nocorr_rda_offset.txt

wrote rda list to: /home/b.lind/offsets/run_20220919_tutorial/rda/offset_outfiles/tutorial_pooled_TRUE_ntraits-6_nocorr_rda_list.RDS

In [8]:
read.table('/home/b.lind/offsets/run_20220919_tutorial/rda/offset_outfiles/tutorial_pooled_NEUTRAL_ntraits-6_nocorr_rda_offset.txt')

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,⋯,X91,X92,X93,X94,X95,X96,X97,X98,X99,X100
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.0000000,0.1594952,0.5837765,1.0337425,1.0166167,0.8685861,0.2675795,0.4595386,0.5667137,1.1396991,⋯,1.4163243,1.3922357,1.3026248,1.5201294,2.0227084,2.3686892,2.4088681,2.5367321,3.0451908,3.0962409
2,0.1594952,0.0000000,0.4452296,0.9403470,0.9616723,0.8995256,0.3600609,0.5651982,0.6156561,1.1937926,⋯,1.4168360,1.4581600,1.3874512,1.5942986,2.0811375,2.4226778,2.4592730,2.5891323,3.1018261,3.1508661
3,0.5837765,0.4452296,0.0000000,0.5761873,0.7165698,0.9281769,0.6835505,0.8720183,0.7687237,1.2588069,⋯,1.2872126,1.5333641,1.5293849,1.6940075,2.1088957,2.4245898,2.4439357,2.5720702,3.0906534,3.1329023
4,1.0337425,0.9403470,0.5761873,0.0000000,0.3079565,0.8339577,1.0193908,1.1274423,0.8914912,1.1210975,⋯,0.8933803,1.3690304,1.4543097,1.5412537,1.8300324,2.0984822,2.0963320,2.2175469,2.7278898,2.7619601
5,1.0166167,0.9616723,0.7165698,0.3079565,0.0000000,0.5678983,0.9335795,0.9823682,0.7110203,0.8267524,⋯,0.6111399,1.0689204,1.1680905,1.2384696,1.5252998,1.8017934,1.8050877,1.9298964,2.4433413,2.4797450
6,0.8685861,0.8995256,0.9281769,0.8339577,0.5678983,0.0000000,0.6722243,0.5912666,0.3297540,0.3378675,⋯,0.6119288,0.6101257,0.6369231,0.7688977,1.1998297,1.5342735,1.5679363,1.6991877,2.2132289,2.2608284
7,0.2675795,0.3600609,0.6835505,1.0193908,0.9335795,0.6722243,0.0000000,0.2401414,0.3597828,0.9247633,⋯,1.2472628,1.1587489,1.0606777,1.2893701,1.8022454,2.1533814,2.1969295,2.3233124,2.8301099,2.8824281
8,0.4595386,0.5651982,0.8720183,1.1274423,0.9823682,0.5912666,0.2401414,0.0000000,0.2860715,0.7746430,⋯,1.1959799,0.9992036,0.8767367,1.1051620,1.6339714,1.9893854,2.0422330,2.1717206,2.6686699,2.7227048
9,0.5667137,0.6156561,0.7687237,0.8914912,0.7110203,0.3297540,0.3597828,0.2860715,0.0000000,0.5861787,⋯,0.9293035,0.8498190,0.7913479,0.9835681,1.4729895,1.8196960,1.8628360,1.9950537,2.5033370,2.5536704
